In [2]:
import numpy as np

from robo.fmin import bayesian_optimization
def objective_function(x):
    y = np.sin(3 * x[0]) * 4 * (x[0] - 1) * (x[0] + 2)
    return y

lower = np.array([0])
upper = np.array([6])

In [3]:

# GP solution
results = bayesian_optimization(objective_function, lower, upper, num_iterations=50)

In [ ]:

# bohamiann
from robo.fmin import bohamiann
results = bohamiann(objective_function, lower, upper, num_iterations=50)


In [ ]:

# Benchmarks
from hpolib.benchmarks.synthetic_functions import Branin
f = Branin()
info = f.get_meta_information()
bounds = np.array(info['bounds'])
results = bayesian_optimization(f, bounds[:, 0], bounds[:, 1], num_iterations=50)


In [7]:

# Training BNN 
import matplotlib.pyplot as plt
import numpy as np

from robo.models.bnn import BayesianNeuralNetwork
from robo.initial_design.init_random_uniform import init_random_uniform

rng = np.random.RandomState(np.random.randint(0, 10000))

def f(x):
    return np.sinc(x * 10 - 5).sum(axis=1)

X = init_random_uniform(np.zeros(1), np.ones(1), 20, rng)
y = f(X)

model = BayesianNeuralNetwork(sampling_method="sghmc",
                              l_rate=np.sqrt(1e-4),
                              mdecay=0.05,
                              burn_in=3000,
                              n_iters=50000,
                              precondition=True,
                              normalize_input=True,
                              normalize_output=True)
model.train(X, y)

x = np.linspace(0, 1, 100)[:, None]
mean_pred, var_pred = model.predict(x)


... compiling theano function
... compiling theano function
